In [1]:
import os
import gzip
import shutil
import json
import pandas as pd
import _pickle as pickle
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline

In [2]:
def transfer_dataframe(patient_id, file_name):
    with open(file_name) as file:
        j_object = json.load(file)
        df = pd.DataFrame(j_object)
        df['datetime'] = pd.to_datetime(df['timestamp'], utc=True) 
        df['pid'] = patient_id
        df_result = df.sort_values(by=['datetime'], ascending = True)
    return df_result

In [3]:
patient_id = 14092221
df_treats = transfer_dataframe(patient_id, '/Users/wang/data/OpenAPS/14092221/direct-sharing-31/treatments.json')

In [4]:
df_entries = transfer_dataframe(patient_id, '/Users/wang/data/OpenAPS/14092221/direct-sharing-31/entries.json')

In [5]:
len(df_entries['sgv'])

73790

In [6]:
df_treats.columns

Index(['_body', '_date', '_description', '_head', '_id', '_type', 'absolute', 'amount', 'appended', 'bg', 'bgcolor', 'bolus', 'carbs', 'changed', 'created_at', 'duration', 'endmills', 'enteredBy', 'enteredinsulin', 'eventType', 'fixed', 'glucose', 'glucoseType', 'insulin', 'maxBasal', 'medtronic', 'notes', 'percent', 'preBolus', 'profile', 'rate', 'ratio', 'raw_duration', 'raw_rate', 'reason', 'relative', 'splitExt', 'splitNow', 'square', 'stale', 'tail', 'targetBottom', 'targetTop', 'timestamp', 'type', 'units', 'wizard', 'datetime', 'pid'], dtype='object')

## Eventtypes

In [7]:
df_treats.eventType.unique()

array(['Note', 'Temp Basal', 'Meal Bolus', 'Correction Bolus', 'BG Check',
       'Carb Correction', '<none>', 'Temporary Target', 'Site Change',
       'Sensor Start', 'OpenAPS Offline', 'Snack Bolus', 'Insulin Change',
       'Sensor Change', 'D.A.D. Alert', 'Combo Bolus', 'Exercise',
       'Question'], dtype=object)

In [25]:
df_treats['bg'].describe()

count     412
unique     96
top       6.2
freq       18
Name: bg, dtype: object

In [8]:
df_treats.set_index('datetime')

_body       _date                                       _description _head                       _id    _type  absolute  amount appended   bg bgcolor bolus  carbs changed                 created_at  duration endmills enteredBy enteredinsulin         eventType  fixed  glucose glucoseType  insulin  maxBasal                                        medtronic                                              notes  percent  preBolus profile   rate ratio                                       raw_duration                                           raw_rate reason  relative splitExt splitNow square stale  tail  targetBottom  targetTop                  timestamp type units wizard       pid
datetime                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
2016-10-05 06:00:00+00:00  050c00e80000000003ac03ac6400000000000000000000...        a410  old6c 2016-10-05T00:00:00 head[1], body[35] op...    6c  57f5d0e58b4c0fc6575ce328    old6c       NaN     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-10-05T00:00:00-06:00       NaN      NaN    CSK26Z            NaN              Note    NaN      NaN         NaN      NaN       NaN       mm://openaps/mm-format-ns-treatments/old6c                                            old6c        NaN       NaN     NaN    NaN   NaN                                                NaN                                                NaN    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-10-05T00:00:00-06:00  NaN   NaN    NaN  14092221
2016-11-06 00:04:45+00:00                                                NaN         NaN                                                NaN   NaN  581ec1168b4c0fc6575fbcbe      NaN     1.000     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-11-05T18:04:45-06:00      30.0      NaN    KHFC9K            NaN        Temp Basal    NaN      NaN         NaN      NaN       NaN  mm://openaps/mm-format-ns-treatments/Temp Basal                                                NaN      NaN       NaN     NaN      1   NaN  {'_type': 'TempBasalDuration', 'duration (min)...  {'_type': 'TempBasal', '_head': '3328', 'temp'...    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-05T18:04:45-06:00  NaN   NaN    NaN  14092221
2016-11-06 00:04:47+00:00                                                NaN         NaN                                                NaN   NaN  581ec1168b4c0fc6575fbcbd      NaN     1.000     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-11-05T18:04:47-06:00      30.0      NaN    KHFC9K            NaN        Temp Basal    NaN      NaN         NaN      NaN       NaN  mm://openaps/mm-format-ns-treatments/Temp Basal                                                NaN      NaN       NaN     NaN      1   NaN  {'_type': 'TempBasalDuration', 'duration (min)...  {'_type': 'TempBasal', '_head': '3328', 'temp'...    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-05T18:04:47-06:00  NaN   NaN    NaN  14092221
2016-11-06 00:19:59+00:00                                                NaN         NaN                                                NaN   NaN  581ec1168b4c0fc6575fbcbc      NaN     0.000     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-11-05T18:19:59-06:00      30.0      NaN    KHFC9K    

In [9]:
df_treats['eventType'].value_counts()

Temp Basal          9444
Meal Bolus           815
Carb Correction      526
Note                 497
BG Check             469
Correction Bolus     291
Snack Bolus          165
Temporary Target     104
Site Change           76
<none>                41
Sensor Start           9
D.A.D. Alert           5
Sensor Change          4
Combo Bolus            4
Insulin Change         3
OpenAPS Offline        2
Exercise               2
Question               1
Name: eventType, dtype: int64

## Boluses

In [10]:
df_treats[df_treats.eventType == 'Temp Basal'].head(5)

_body _date _description _head                       _id _type  absolute  amount appended   bg bgcolor bolus  carbs changed                 created_at  duration endmills enteredBy enteredinsulin   eventType  fixed  glucose glucoseType  insulin  maxBasal                                        medtronic notes  percent  preBolus profile rate ratio                                       raw_duration                                           raw_rate reason  relative splitExt splitNow square stale  tail  targetBottom  targetTop                  timestamp type units wizard                  datetime       pid
11293   NaN   NaN          NaN   NaN  581ec1168b4c0fc6575fbcbe   NaN       1.0     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-11-05T18:04:45-06:00      30.0      NaN    KHFC9K            NaN  Temp Basal    NaN      NaN         NaN      NaN       NaN  mm://openaps/mm-format-ns-treatments/Temp Basal   NaN      NaN       NaN     NaN    1   NaN  {'_type': 'TempBasalDuration', 'duration (min)...  {'_type': 'TempBasal', '_head': '3328', 'temp'...    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-05T18:04:45-06:00  NaN   NaN    NaN 2016-11-06 00:04:45+00:00  14092221
11292   NaN   NaN          NaN   NaN  581ec1168b4c0fc6575fbcbd   NaN       1.0     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-11-05T18:04:47-06:00      30.0      NaN    KHFC9K            NaN  Temp Basal    NaN      NaN         NaN      NaN       NaN  mm://openaps/mm-format-ns-treatments/Temp Basal   NaN      NaN       NaN     NaN    1   NaN  {'_type': 'TempBasalDuration', 'duration (min)...  {'_type': 'TempBasal', '_head': '3328', 'temp'...    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-05T18:04:47-06:00  NaN   NaN    NaN 2016-11-06 00:04:47+00:00  14092221
11290   NaN   NaN          NaN   NaN  581ec1168b4c0fc6575fbcbc   NaN       0.0     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-11-05T18:19:59-06:00      30.0      NaN    KHFC9K            NaN  Temp Basal    NaN      NaN         NaN      NaN       NaN  mm://openaps/mm-format-ns-treatments/Temp Basal   NaN      NaN       NaN     NaN    0   NaN  {'_type': 'TempBasalDuration', 'duration (min)...  {'_type': 'TempBasal', '_head': '3300', 'temp'...    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-05T18:19:59-06:00  NaN   NaN    NaN 2016-11-06 00:19:59+00:00  14092221
11288   NaN   NaN          NaN   NaN  581ec1168b4c0fc6575fbcb9   NaN       1.0     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-11-05T20:32:53-06:00      30.0      NaN    KHFC9K            NaN  Temp Basal    NaN      NaN         NaN      NaN       NaN  mm://openaps/mm-format-ns-treatments/Temp Basal   NaN      NaN       NaN     NaN    1   NaN  {'_type': 'TempBasalDuration', 'duration (min)...  {'_type': 'TempBasal', '_head': '3328', 'temp'...    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-05T20:32:53-06:00  NaN   NaN    NaN 2016-11-06 02:32:53+00:00  14092221
11287   NaN   NaN          NaN   NaN  581ec1168b4c0fc6575fbcbb   NaN       1.0     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-11-05T20:36:09-06:00      30.0      NaN    KHFC9K            NaN  Temp Basal    NaN      NaN         NaN      NaN       NaN  mm://openaps/mm-format-ns-treatments/Temp Basal   NaN      NaN       NaN     NaN    1   NaN  {'_type': 'TempBasalDuration', 'duration (min)...  {'_type': 'TempBasal', '_head': '3328', 'temp'...    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-05T20:36:09-06:00  NaN   NaN    NaN 2016-11-06 02:36:09+00:00  14092221

In [11]:
df_treats[df_treats.eventType == 'Meal Bolus'].head(5)

_body _date _description _head                       _id _type  absolute  amount appended   bg bgcolor                                              bolus  carbs changed                 created_at  duration endmills enteredBy enteredinsulin   eventType  fixed  glucose  glucoseType  insulin  maxBasal                                        medtronic                                              notes  percent  preBolus profile rate ratio raw_duration raw_rate reason  relative splitExt splitNow square stale  tail  targetBottom  targetTop                  timestamp type units                                             wizard                  datetime       pid
11090   NaN   NaN          NaN   NaN  582a6e2d8b4c0fc6576097c1   NaN       NaN     NaN      NaN  6.6     NaN  {'_type': 'Bolus', 'unabsorbed': 0, '_head': '...  114.0     NaN  2016-11-14T18:03:06-07:00       0.0      NaN    KHFC9K            NaN  Meal Bolus    NaN      6.6  BolusWizard      9.7       NaN  mm://openaps/mm-format-ns-treatments/Meal Bolus  Normal bolus with wizard.\nProgrammed bolus 9....      NaN       NaN     NaN  NaN    12          NaN      NaN    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-14T18:03:06-07:00  NaN   NaN  {'_type': 'BolusWizard', 'sensitivity': 4.8, '... 2016-11-15 01:03:06+00:00  14092221
11088   NaN   NaN          NaN   NaN  582a6e2d8b4c0fc6576097c3   NaN       NaN     NaN      NaN  7.4     NaN  {'_type': 'Bolus', 'unabsorbed': 8, '_head': '...   16.0     NaN  2016-11-14T18:47:10-07:00       0.0      NaN    KHFC9K            NaN  Meal Bolus    NaN      7.4  BolusWizard      1.3       NaN  mm://openaps/mm-format-ns-treatments/Meal Bolus  Normal bolus with wizard.\nProgrammed bolus 1....      NaN       NaN     NaN  NaN    12          NaN      NaN    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-14T18:47:10-07:00  NaN   NaN  {'_type': 'BolusWizard', 'sensitivity': 4.8, '... 2016-11-15 01:47:10+00:00  14092221
11057   NaN   NaN          NaN   NaN  582ba2838b4c0fc65760ad12   NaN       NaN     NaN      NaN    7     NaN  {'_type': 'Bolus', 'unabsorbed': 0.3, '_head':...   51.0     NaN   2016-11-15T18:57:36.902Z       0.0      NaN    FFMUF3            NaN  Meal Bolus    NaN      7.0  BolusWizard      4.6       NaN  mm://openaps/mm-format-ns-treatments/Meal Bolus  Normal bolus with wizard.\nProgrammed bolus 4....      NaN       NaN     NaN  NaN    11          NaN      NaN    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-15T11:57:51-07:00  NaN   NaN  {'_type': 'BolusWizard', 'sensitivity': 4.8, '... 2016-11-15 18:57:51+00:00  14092221
11054   NaN   NaN          NaN   NaN  582bb2c28b4c0fc65760ae2b   NaN       NaN     NaN      NaN  7.5     NaN  {'_type': 'Bolus', 'unabsorbed': 0, '_head': '...   75.0     NaN   2016-11-16T00:56:38.143Z       0.0      NaN    FFMUF3            NaN  Meal Bolus    NaN      7.5  BolusWizard      6.5       NaN  mm://openaps/mm-format-ns-treatments/Meal Bolus  Normal bolus with wizard.\nProgrammed bolus 6....      NaN       NaN     NaN  NaN    12          NaN      NaN    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-15T17:56:21-07:00  NaN   NaN  {'_type': 'BolusWizard', 'sensitivity': 4.8, '... 2016-11-16 00:56:21+00:00  14092221
11038   NaN   NaN          NaN   NaN  582ceeaa8b4c0fc65760c2f8   NaN       NaN     NaN      NaN  4.7     NaN  {'_type': 'Bolus', 'unabsorbed': 0.6, '_head':...   10.0     NaN  2016-11-16T10:53:42-07:00       0.0      NaN    FFMUF3            NaN  Meal Bolus    NaN      4.7  BolusWizard      0.9       NaN  mm://openaps/mm-format-ns-treatments/Meal Bolus  Normal bolus with wizard.\nProgrammed bolus 0....      NaN       NaN     NaN  NaN    11          NaN      NaN    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-16T10:53:42-07:00  NaN   NaN  {'_type': 'BolusWizard', 'sensitivity': 4.8, '... 2016-11-16 17:53:42+00

In [12]:
df_treats[df_treats.eventType == 'Snack Bolus'].head(5)

_body _date _description _head                       _id _type  absolute  amount appended   bg bgcolor bolus  carbs changed                created_at  duration endmills enteredBy enteredinsulin    eventType  fixed  glucose glucoseType  insulin  maxBasal medtronic                                              notes  percent  preBolus profile rate ratio raw_duration raw_rate reason  relative splitExt splitNow square stale  tail  targetBottom  targetTop timestamp type units wizard datetime       pid
4160   NaN   NaN          NaN   NaN  586c3fac1c9764ba303935aa   NaN       NaN     NaN      NaN  NaN     NaN   NaN    6.0     NaN  2017-01-04T00:19:56.865Z       0.0      NaN    OV7ZA3            NaN  Snack Bolus    NaN      NaN         NaN      NaN       NaN       NaN                                                NaN      NaN       NaN     NaN  NaN   NaN          NaN      NaN              NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN       NaN  NaN   NaN    NaN      NaT  14092221
4723   NaN   NaN          NaN   NaN  586882661c9764ba3038fa93   NaN       NaN     NaN      NaN  NaN     NaN   NaN   20.0     NaN  2017-01-01T03:18:00.000Z       0.0      NaN    CNPUVF            NaN  Snack Bolus    NaN      6.2      Sensor      NaN       NaN       NaN  Dropping already. Extra carbs w/o insulin for ...      NaN       NaN     NaN  NaN   NaN          NaN      NaN              NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN       NaN  NaN  mmol    NaN      NaT  14092221
6536   NaN   NaN          NaN   NaN  585ae82a1c9764ba3037bb86   NaN       NaN     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-12-21T17:33:00.000Z       0.0      NaN    OV7ZA3            NaN  Snack Bolus    NaN      7.2      Finger      0.4       NaN       NaN                                                NaN      NaN       NaN     NaN  NaN   NaN          NaN      NaN              NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN       NaN  NaN  mmol    NaN      NaT  14092221
6636   NaN   NaN          NaN   NaN  5859a3041c9764ba30378cf4   NaN       NaN     NaN      NaN  NaN     NaN   NaN   15.0     NaN  2016-12-20T20:59:00.000Z       0.0      NaN    XLCB9D            NaN  Snack Bolus    NaN      6.4      Finger      1.0       NaN       NaN                                                NaN      NaN       NaN     NaN  NaN   NaN          NaN      NaN              NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN       NaN  NaN  mmol    NaN      NaT  14092221
8674   NaN   NaN          NaN   NaN  584604931c9764ba303568b5   NaN       NaN     NaN      NaN  NaN     NaN   NaN    7.0     NaN  2016-12-06T00:15:00.000Z       0.0      NaN    OV7ZA3            NaN  Snack Bolus    NaN      6.5      Finger      0.5       NaN       NaN                                                NaN      NaN       NaN     NaN  NaN   NaN          NaN      NaN              NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN       NaN  NaN  mmol    NaN      NaT  14092221

In [13]:
df_treats[df_treats.eventType == 'Combo Bolus'].head(5)

_body _date _description _head                       _id _type  absolute  amount appended   bg bgcolor bolus  carbs changed                created_at  duration endmills enteredBy enteredinsulin    eventType  fixed  glucose glucoseType  insulin  maxBasal medtronic                                              notes  percent  preBolus profile rate ratio raw_duration raw_rate reason  relative splitExt splitNow square stale  tail  targetBottom  targetTop timestamp type units wizard datetime       pid
11793   NaN   NaN          NaN   NaN  57e6e73458d2ad88ea5b43fc   NaN       NaN     NaN      NaN  NaN     NaN   NaN   33.0     NaN  2016-09-24T20:30:00.000Z      30.0      NaN    CNPUVF            1.7  Combo Bolus    NaN      7.0      Finger    1.700       NaN       NaN  1st October pizza done with regular bolus. 2nd...      NaN       NaN     NaN  NaN   NaN          NaN      NaN              NaN        0      100    NaN   NaN   NaN           NaN        NaN       NaN  NaN  mmol    NaN      NaT  14092221
12200   NaN   NaN          NaN   NaN  57b1139f58d2ad88ea57a5c2   NaN       NaN     NaN      NaN  NaN     NaN   NaN   74.0     NaN  2016-08-15T00:58:00.000Z       0.0      NaN    OV7ZA3            NaN  Combo Bolus    NaN      5.1      Sensor    4.100       NaN       NaN  2.0 given at initial dosing. 2.0 given 60 minu...      NaN       NaN          NaN   NaN          NaN      NaN              NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN       NaN  NaN  mmol    NaN      NaT  14092221
12211   NaN   NaN          NaN   NaN  57afcba858d2ad88ea57869f   NaN       NaN     NaN      NaN  NaN     NaN   NaN   45.0     NaN  2016-08-14T01:27:00.000Z      60.0      NaN    OV7ZA3           2.25  Combo Bolus    NaN      8.4      Sensor    2.250       NaN       NaN  Initial insulin of 44 given for pizza.  2nd bo...      NaN       NaN     NaN  NaN   NaN          NaN      NaN              NaN        0      100    NaN   NaN   NaN           NaN        NaN       NaN  NaN  mmol    NaN      NaT  14092221
12450   NaN   NaN          NaN   NaN  5770970f58d2ad88ea5337a1   NaN       NaN     NaN      NaN  NaN     NaN   NaN  106.0     NaN  2016-06-26T18:30:00.000Z      90.0      NaN    OV7ZA3            5.5  Combo Bolus    NaN      9.2      Finger    3.575       NaN       NaN  3.5 units of insulin given.  Balance of 2 unit...      NaN       NaN     NaN  NaN   NaN          NaN      NaN         1.283333       35       65    NaN   NaN   NaN           NaN        NaN       NaN  NaN  mmol    NaN      NaT  14092221

## Carb

In [14]:
df_treats[df_treats.eventType == 'Carb Correction'].head(5)

_body _date _description _head                       _id _type  absolute  amount appended    bg bgcolor bolus  carbs changed                 created_at  duration endmills enteredBy enteredinsulin        eventType  fixed  glucose  glucoseType  insulin  maxBasal                                          medtronic                                              notes  percent  preBolus profile rate ratio raw_duration raw_rate reason  relative splitExt splitNow square stale  tail  targetBottom  targetTop                  timestamp type units                                             wizard                  datetime       pid
10831   NaN   NaN          NaN   NaN  5831dead8b4c0fc657611aea   NaN       NaN     NaN      NaN   5.3     NaN   NaN   48.0     NaN  2016-11-20T10:29:12-07:00       NaN      NaN    FFMUF3            NaN  Carb Correction    NaN      5.3  BolusWizard      NaN       NaN  mm://openaps/mm-format-ns-treatments/Carb Corr...  Food estimate 4.3\nCorrection estimate 0\nBolu...      NaN       NaN     NaN  NaN    11          NaN      NaN    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-20T10:29:12-07:00  NaN   NaN  {'_type': 'BolusWizard', 'sensitivity': 4.8, '... 2016-11-20 17:29:12+00:00  14092221
10734   NaN   NaN          NaN   NaN  5832a5b28b4c0fc657612960   NaN       NaN     NaN      NaN   4.5     NaN   NaN    9.0     NaN  2016-11-21T00:38:29-07:00       NaN      NaN    FFMUF3            NaN  Carb Correction    NaN      4.5  BolusWizard      NaN       NaN  mm://openaps/mm-format-ns-treatments/Carb Corr...  Food estimate 0.6\nCorrection estimate 51.1\nB...      NaN       NaN     NaN  NaN    13          NaN      NaN    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-21T00:38:29-07:00  NaN   NaN  {'_type': 'BolusWizard', 'sensitivity': 4.8, '... 2016-11-21 07:38:29+00:00  14092221
9606    NaN   NaN          NaN   NaN  583d950c8b4c0fc65761f40f   NaN       NaN     NaN      NaN   6.2     NaN   NaN   23.0     NaN  2016-11-29T07:45:32-07:00       NaN      NaN    FFMUF3            NaN  Carb Correction    NaN      6.2  BolusWizard      NaN       NaN  mm://openaps/mm-format-ns-treatments/Carb Corr...  Food estimate 2.5\nCorrection estimate 0.1\nBo...      NaN       NaN     NaN  NaN     9          NaN      NaN    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-11-29T07:45:32-07:00  NaN   NaN  {'_type': 'BolusWizard', 'sensitivity': 3.9, '... 2016-11-29 14:45:32+00:00  14092221
9333    NaN   NaN          NaN   NaN  5840d49f1c9764ba3034eb46   NaN       NaN     NaN      NaN  10.5     NaN   NaN   55.0     NaN  2016-12-01T18:49:17-07:00       NaN      NaN    FFMUF3            NaN  Carb Correction    NaN     10.5  BolusWizard      NaN       NaN  mm://openaps/mm-format-ns-treatments/Carb Corr...  Food estimate 4.5\nCorrection estimate 1.1\nBo...      NaN       NaN     NaN  NaN    12          NaN      NaN    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-12-01T18:49:17-07:00  NaN   NaN  {'_type': 'BolusWizard', 'sensitivity': 4, '_h... 2016-12-02 01:49:17+00:00  14092221
8249    NaN   NaN          NaN   NaN  584abcf71c9764ba3035f25a   NaN       NaN     NaN      NaN   7.6     NaN   NaN   33.0     NaN  2016-12-09T07:13:42-07:00       NaN      NaN    FFMUF3            NaN  Carb Correction    NaN      7.6  BolusWizard      NaN       NaN  mm://openaps/mm-format-ns-treatments/Carb Corr...  Food estimate 3.7\nCorrection estimate 0.4\nBo...      NaN       NaN     NaN  NaN   8.8          NaN      NaN    NaN       NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN  2016-12-09T07:13:42-07:00  NaN   NaN  {'_type': 'BolusWizard', 'sensitivity': 4, '_h... 2016-12-09 14:13:42+00:00  14092221

In [15]:
df_treats[df_treats.eventType == 'Carb Correction']['notes'].values[:5]

array(['Food estimate 4.3\nCorrection estimate 0\nBolus estimate 4.3\nTarget low 4.6\nTarget high 5.6\nHypothetical glucose delta 0\nGlucose was: 5.3',
       'Food estimate 0.6\nCorrection estimate 51.1\nBolus estimate 0.5\nTarget low 5\nTarget high 6.5\nHypothetical glucose delta 0\nGlucose was: 4.5',
       'Food estimate 2.5\nCorrection estimate 0.1\nBolus estimate 2.6\nTarget low 5\nTarget high 5.6\nHypothetical glucose delta 0\nGlucose was: 6.2',
       'Food estimate 4.5\nCorrection estimate 1.1\nBolus estimate 5.6\nTarget low 6\nTarget high 6\nHypothetical glucose delta 0\nGlucose was: 10.5',
       'Food estimate 3.7\nCorrection estimate 0.4\nBolus estimate 4.1\nTarget low 6\nTarget high 6\nHypothetical glucose delta 0\nGlucose was: 7.6'],
      dtype=object)

## Others

In [16]:
df_treats['glucoseType'].value_counts()

Finger         1469
Sensor          432
BolusWizard     412
Name: glucoseType, dtype: int64

In [17]:
df_treats[df_treats.eventType == 'Exercise']

_body _date _description _head                       _id _type  absolute  amount appended   bg bgcolor bolus  carbs changed                created_at  duration endmills enteredBy enteredinsulin eventType  fixed  glucose glucoseType  insulin  maxBasal medtronic                                           notes  percent  preBolus profile rate ratio raw_duration raw_rate reason  relative splitExt splitNow square stale  tail  targetBottom  targetTop timestamp type units wizard datetime       pid
12194   NaN   NaN          NaN   NaN  57b24a1958d2ad88ea57be19   NaN       NaN     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-08-15T19:15:00.000Z     180.0      NaN    OV7ZA3            NaN  Exercise    NaN      NaN         NaN      NaN       NaN       NaN  At lake for 3 hours.  Active play, hot outside      NaN       NaN          NaN   NaN          NaN      NaN              NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN       NaN  NaN   NaN    NaN      NaT  14092221
12453   NaN   NaN          NaN   NaN  5770898c58d2ad88ea5336c8   NaN       NaN     NaN      NaN  NaN     NaN   NaN    NaN     NaN  2016-06-26T16:00:00.000Z     120.0      NaN    CNPUVF            NaN  Exercise    NaN      NaN         NaN      NaN       NaN       NaN         Extreme Air Trampoline park. Bday party      NaN       NaN     NaN  NaN   NaN          NaN      NaN              NaN      NaN      NaN    NaN   NaN   NaN           NaN        NaN       NaN  NaN   NaN    NaN      NaT  14092221

## Integerate into one time line